**Auto đọc từ file text.**

In [ ]:
# prompt: chọn đầu vào là file text, chia file text thành nhiều đoạn mỗi đoạn 1429 từ. sau đó chuyển văn bản thành giọng nói. đặt tên file theo tên file text

from google.colab import files
from google import genai
import contextlib
import wave
from google.colab import userdata

# Get API key
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize the client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Select a model
MODEL_ID = "gemini-2.5-flash-preview-tts" # @param ["gemini-2.5-flash-preview-tts","gemini-2.5-pro-preview-tts"] {"allow-input":true, isTemplate: true}

# Choose a voice
voice_name = "Charon" # @param ["Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda", "Orus", "Aoede", "Callirhoe", "Autonoe", "Enceladus", "Iapetus", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalgethi", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafar"]

# Helper function to create a wave file
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width) # Corrected to setsampwidth
        wf.setframerate(rate)
        yield wf

# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=1429):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Upload the text file
uploaded = files.upload()

for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()

    chunks = split_text_into_chunks(text)
    base_filename = filename.rsplit('.', 1)[0]

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}")
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=f"""
              Speak in a warm, experienced voice, in Vietnamese:
              "{chunk}"

            """,
            config={
                "response_modalities": ['Audio'],
                "speech_config": {
                    "voice_config": {
                        "prebuilt_voice_config": {
                            "voice_name": voice_name
                        }
                    }
                }
            },
        )

        # Get the audio blob
        if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
            audio_blob = response.candidates[0].content.parts[0].inline_data

            # Define the output filename
            output_fname = f'{base_filename}_part_{i+1}.wav'

            # Write the audio data to a wave file
            with wave_file(output_fname) as wav:
                wav.writeframes(audio_blob.data)

            print(f"Saved chunk {i+1} as {output_fname}")

            # Download the audio file
            files.download(output_fname)
        else:
            print(f"Skipping chunk {i+1} as no audio was generated.")